In [1]:
!pwd

/home/techie/Desktop/general/End-to-End-DL/notebooks


In [2]:
import os

os.chdir("../")
!pwd

/home/techie/Desktop/general/End-to-End-DL


In [3]:
import tensorflow as tf

model = tf.keras.models.load_model("artifacts/training/model.h5")

2023-10-09 17:13:06.759685: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 17:13:10.529651: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-10-09 17:13:10.558925: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-09 17:13:21.159993: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [6]:
from pathlib import Path
from PnemoniaDisease.utils.utils import read_yaml, save_json
from PnemoniaDisease.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from PnemoniaDisease import logger

In [9]:
class Evaluation:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        self.params = read_yaml(params_filepath)
        self.config = read_yaml(config_filepath)
    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.params.IMAGE_SIZE[:-1],
            batch_size=self.params.BATCH_SIZE,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        training_data = os.path.join("data/processed", "train")
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=Path(training_data),
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.training.trained_model_path)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [10]:
try:
    evaluation = Evaluation(config_filepath=CONFIG_FILE_PATH)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
   raise e

[2023-10-09 18:05:00,517: INFO: utils: yaml file: params.yaml loaded successfully]
[2023-10-09 18:05:00,585: INFO: utils: yaml file: config/config.yaml loaded successfully]


Found 119 images belonging to 2 classes.
8/8 [==============================] - 25s 2s/step - loss: 0.5842 - accuracy: 0.7479
[2023-10-09 18:05:36,720: INFO: utils: json file saved at: scores.json]
